In [4]:
import sys
import glob
import pickle
import numpy as np
from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Bidirectional, Dense, Dropout, LSTM, Activation, TimeDistributed, Embedding
from keras.callbacks import ModelCheckpoint, TensorBoard, History, Callback
from keras.utils import np_utils
import re 
import pandas as pd
import music21
import IPython
from tqdm import tqdm
from playsound import playsound
from datetime import datetime
from music21 import converter, instrument, note, chord, stream
import json

In [5]:
#Get the midi files
midi_files = '/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/*.mid'
timestamp = re.sub(r'[-: ]','',str(datetime.now()).split('.')[0])[:-2]
output_name = midi_files.split('/')[-2]
output_tag = '/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/output/{}-{}-'.format(timestamp, output_name)
songs = glob.glob(midi_files)

#Specify how many songs to take for training
songs = songs[150:550]

In [6]:
# Create function get_notes() that will return a list of notes and chords present in the .mid file.

def get_notes():
        # convert midi file dataset to notes
    notes = [] # list of notes and chords
    note_count = 0
    print("\n**Loading Midi files**")
    for file in songs: # loading midi filepaths
        print(file)
        try:
            midi = converter.parse(file) # midi type music21.stream.Score
            parts = instrument.partitionByInstrument(midi)

            if parts:
                notes_to_parse = parts.parts[0].recurse()
            else:
                notes_to_parse = midi.flat.notes
            # notes_to_parse type music21.stream.iterator.RecursiveIterator
            for e in notes_to_parse:
                if isinstance(e, note.Note):
                    notes.append(str(e.pitch))
                elif isinstance(e, chord.Chord):
                    to_append = '.'.join(str(n) for n in e.normalOrder)
                    notes.append(to_append)
            note_count +=1
        except Exception as e:
            print(e)
            pass
    n_vocab = len(set(notes))
    print("Loaded {} midi files {} notes and {} unique notes".format(len(songs), len(notes), n_vocab))

    note_file = output_tag + 'input_notes'
    with open(note_file, 'wb') as f:
        pickle.dump(notes, f)
    print("Input notes/chords stored as {} then pickled at {}".format(type(notes), note_file))
    print("First 20 notes/chords: {}".format(notes[:20]))
    return notes

In [7]:
get_notes()


**Loading Midi files**
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/MuskratRamble.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/HowHighTheMoon (1).mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Sidewalk.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/OhSherrie.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Bluesette.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Bebop.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/LithiumSunset.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/HesMisstraKnow-it-all.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/JamaicanNights.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/DontknowMuch.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/LostWithoutYou.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/KingKongItself.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/WhosCryinNow.mid
/Users/a

/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Crazylittlethingcalllove.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Funkytown.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/GhostOfaChance.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Dying.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/PrettyWoman.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/IllNeverSmileAgain.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Morning (1).mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Guilty.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/ZootAllures.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/MakeLove.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/MisterSandman.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/OpenedTheDoor.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/AlJolsonMedley.mid
/Users/anant.a.sharma/Desktop/Darya/A

/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/UncleMeat.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/BackToBlack (1).mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/SecondMarriage.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Gemini.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/TunnelVision.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/BigTime.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/LastNightIDidntGetToSleepAtAll.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Moondance (1).mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/ItsNotforMeToSay.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Fresh.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/ImAFoolToWantYou (1).mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Blueocean.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/TonightICelebrateMyLove.mid
/Users/anant.a.

/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/CountryDoctor.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/NativeSonsOfADistantLand.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Atavachron.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/EyesOfMind.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Montana.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/SweetAboutMe.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/DontDreamOfAnybodyButMe.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/YouAreTheSunshineOfMyLife(2).mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/TheSunGoesDown.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/RoadWarriors.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/SavetheLastDanceforme.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/AllIDoIsDreamOfYou.mid
/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/Jazz/Lights.

['C4',
 'G#2',
 'C4',
 'E-3',
 'C4',
 'C3',
 'C4',
 'G#2',
 'C#4',
 'B-2',
 'C#4',
 'B-2',
 'C#4',
 'C#3',
 'C#4',
 'C#4',
 'B-2',
 'G#2',
 'C#4',
 'B-2',
 'C#4',
 'E-2',
 'C#4',
 'B-2',
 'C#4',
 'G2',
 'E-2',
 'C#4',
 'C#4',
 'G#2',
 'C#4',
 'C3',
 'C#4',
 'E-3',
 'C#4',
 'C3',
 'E-2',
 'C4',
 'G#2',
 'C4',
 'C3',
 'C4',
 'E-3',
 'B3',
 'C4',
 'C3',
 'B3',
 'C4',
 'G#2',
 'C4',
 'G2',
 'E-2',
 'G2',
 'F2',
 'D4',
 'G2',
 'D4',
 'D4',
 'G2',
 'D4',
 'G2',
 'C4',
 'C3',
 'C4',
 'G2',
 'C4',
 'C#4',
 'E-2',
 'E-4',
 'C#4',
 'B-3',
 'C#4',
 'C4',
 'C3',
 'C4',
 'E-3',
 'C4',
 'C3',
 'G#2',
 'B-4',
 'B-2',
 'C5',
 'B-4',
 'B-2',
 'G4',
 'C#3',
 'F4',
 'C#3',
 'C#4',
 'G4',
 'G#4',
 'A4',
 'B-2',
 'B-4',
 'C#3',
 'G#4',
 'G4',
 'B-2',
 'F4',
 'E-4',
 'G2',
 'C#4',
 'C4',
 'G#2',
 'B3',
 'E-2',
 'C4',
 'E-4',
 'G#2',
 'C4',
 'G#4',
 'G2',
 'F2',
 'E-4',
 'C4',
 'F4',
 'C2',
 'E-4',
 'G4',
 'F2',
 'E-4',
 'F4',
 'B-1',
 'F4',
 'F4',
 'F2',
 'F4',
 'B-1',
 'G4',
 'G#4',
 'A4',
 'B-4',
 'G#4',


In [46]:
data_directory = "/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA"
charIndex_json = "/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/char_to_index.json"
model_weights_directory = '../DATA/output/Test3_Weights/'

In [12]:
with open('/Users/anant.a.sharma/Desktop/Darya/AI-Jukebox/DATA/output/201812261056-Jazz-input_notes', 'rb') as filepath:
    notes = pickle.load(filepath)


In [29]:
# prepare_data(notes):
#mapping character to index
char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(notes))))}
print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87

with open(charIndex_json, mode = "w") as f:
    json.dump(char_to_index, f)

index_to_char = {i: ch for (ch, i) in char_to_index.items()}
unique_chars = len(char_to_index)
all_characters = np.asarray([char_to_index[c] for c in notes], dtype = np.int32)
print("Total number of characters = "+str(all_characters.shape[0])) #155222
all_characters, unique_chars

Number of unique characters in our whole tunes database = 1028
Total number of characters = 367003


(array([ 962, 1012,  962, ...,  936,  944,  985], dtype=int32), 1028)

In [16]:
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [32]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i + 1]
        yield X, Y

In [43]:
length = all_characters.shape[0]
batch_chars = int(length / BATCH_SIZE) 

for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 22937, 64)  #it denotes number of batches. It runs everytime when
    #new batch is created. We have a total of 358 batches.
    X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
    Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 1028)
    for batch_index in range(0, 16):  #it denotes each row in a batch.  
        for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
            #each time-step character in a sequence.
            X[batch_index, i] = all_characters[batch_index * batch_chars + start + i]
            Y[batch_index, i, all_characters[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
            #correct label will be the next character in the sequence. So, the next character will be denoted by
            #all_chars[batch_index * batch_chars + start + i + 1]

## Model Architecture & Training

In [60]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(128, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [48]:
def training_model(notes, epochs = 80):
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in notes], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0]))
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Test3_Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../DATA/log.csv", index = False)


In [49]:
training_model(notes, epochs = 80)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (16, 64, 512)             526336    
_________________________________________________________________
lstm_3 (LSTM)                (16, 64, 256)             787456    
_________________________________________________________________
dropout_3 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_4 (LSTM)                (16, 64, 128)             197120    
_________________________________________________________________
dropout_4 (Dropout)          (16, 64, 128)             0         
_________________________________________________________________
time_distributed_2 (TimeDist (16, 64, 1028)            132612    
_________________________________________________________________
activation_2 (Activation)    (16, 64, 1028)            0         
Total para

Batch: 120, Loss: 4.897636413574219, Accuracy: 0.0732421875
Batch: 121, Loss: 4.946328163146973, Accuracy: 0.0810546875
Batch: 122, Loss: 4.990246772766113, Accuracy: 0.0615234375
Batch: 123, Loss: 4.897343635559082, Accuracy: 0.0546875
Batch: 124, Loss: 4.873260498046875, Accuracy: 0.0400390625
Batch: 125, Loss: 4.851483345031738, Accuracy: 0.0498046875
Batch: 126, Loss: 4.862858772277832, Accuracy: 0.0498046875
Batch: 127, Loss: 4.976576328277588, Accuracy: 0.041015625
Batch: 128, Loss: 4.817749500274658, Accuracy: 0.0361328125
Batch: 129, Loss: 4.868465900421143, Accuracy: 0.056640625
Batch: 130, Loss: 5.033564567565918, Accuracy: 0.029296875
Batch: 131, Loss: 5.0700273513793945, Accuracy: 0.0361328125
Batch: 132, Loss: 5.101110935211182, Accuracy: 0.025390625
Batch: 133, Loss: 5.2700653076171875, Accuracy: 0.0107421875
Batch: 134, Loss: 5.254428863525391, Accuracy: 0.0078125
Batch: 135, Loss: 5.1446051597595215, Accuracy: 0.01171875
Batch: 136, Loss: 5.104710102081299, Accuracy: 0.

Batch: 259, Loss: 4.6953630447387695, Accuracy: 0.0625
Batch: 260, Loss: 4.900588035583496, Accuracy: 0.0556640625
Batch: 261, Loss: 4.889856338500977, Accuracy: 0.056640625
Batch: 262, Loss: 5.16555643081665, Accuracy: 0.0341796875
Batch: 263, Loss: 5.252300262451172, Accuracy: 0.033203125
Batch: 264, Loss: 5.278597354888916, Accuracy: 0.041015625
Batch: 265, Loss: 5.115448951721191, Accuracy: 0.029296875
Batch: 266, Loss: 5.2432403564453125, Accuracy: 0.033203125
Batch: 267, Loss: 5.124824523925781, Accuracy: 0.0478515625
Batch: 268, Loss: 5.074643611907959, Accuracy: 0.0341796875
Batch: 269, Loss: 5.045519828796387, Accuracy: 0.0380859375
Batch: 270, Loss: 5.042701721191406, Accuracy: 0.0322265625
Batch: 271, Loss: 4.972190856933594, Accuracy: 0.0341796875
Batch: 272, Loss: 4.908219337463379, Accuracy: 0.0390625
Batch: 273, Loss: 4.814678192138672, Accuracy: 0.0458984375
Batch: 274, Loss: 4.719823360443115, Accuracy: 0.0380859375
Batch: 275, Loss: 4.810360908508301, Accuracy: 0.0419

Batch: 41, Loss: 5.2682905197143555, Accuracy: 0.0400390625
Batch: 42, Loss: 5.358591556549072, Accuracy: 0.0341796875
Batch: 43, Loss: 5.143364906311035, Accuracy: 0.0380859375
Batch: 44, Loss: 5.079538345336914, Accuracy: 0.041015625
Batch: 45, Loss: 4.9537482261657715, Accuracy: 0.041015625
Batch: 46, Loss: 5.010895729064941, Accuracy: 0.044921875
Batch: 47, Loss: 4.989361763000488, Accuracy: 0.064453125
Batch: 48, Loss: 5.088984489440918, Accuracy: 0.03515625
Batch: 49, Loss: 5.029232025146484, Accuracy: 0.0439453125
Batch: 50, Loss: 5.008849143981934, Accuracy: 0.0400390625
Batch: 51, Loss: 4.965741157531738, Accuracy: 0.0458984375
Batch: 52, Loss: 5.051383018493652, Accuracy: 0.0537109375
Batch: 53, Loss: 4.992812156677246, Accuracy: 0.064453125
Batch: 54, Loss: 4.91534948348999, Accuracy: 0.056640625
Batch: 55, Loss: 4.964100360870361, Accuracy: 0.037109375
Batch: 56, Loss: 5.06924295425415, Accuracy: 0.0380859375
Batch: 57, Loss: 5.014641761779785, Accuracy: 0.0390625
Batch: 58

Batch: 182, Loss: 4.883802890777588, Accuracy: 0.0625
Batch: 183, Loss: 4.797506809234619, Accuracy: 0.0693359375
Batch: 184, Loss: 4.855471611022949, Accuracy: 0.0615234375
Batch: 185, Loss: 4.829164028167725, Accuracy: 0.083984375
Batch: 186, Loss: 4.974798202514648, Accuracy: 0.0595703125
Batch: 187, Loss: 5.211356163024902, Accuracy: 0.0712890625
Batch: 188, Loss: 4.761846542358398, Accuracy: 0.060546875
Batch: 189, Loss: 4.783412933349609, Accuracy: 0.0419921875
Batch: 190, Loss: 4.789878845214844, Accuracy: 0.05078125
Batch: 191, Loss: 4.838083744049072, Accuracy: 0.0712890625
Batch: 192, Loss: 4.912627220153809, Accuracy: 0.072265625
Batch: 193, Loss: 4.910173416137695, Accuracy: 0.06640625
Batch: 194, Loss: 5.078155517578125, Accuracy: 0.072265625
Batch: 195, Loss: 4.8829803466796875, Accuracy: 0.0673828125
Batch: 196, Loss: 4.595537185668945, Accuracy: 0.076171875
Batch: 197, Loss: 4.751789093017578, Accuracy: 0.091796875
Batch: 198, Loss: 4.6454668045043945, Accuracy: 0.12597

...

Batch: 154, Loss: 4.8660888671875, Accuracy: 0.056640625
Batch: 155, Loss: 4.730740547180176, Accuracy: 0.056640625
Batch: 156, Loss: 4.873954772949219, Accuracy: 0.068359375
Batch: 157, Loss: 4.9098005294799805, Accuracy: 0.0595703125
Batch: 158, Loss: 4.871298789978027, Accuracy: 0.060546875
Batch: 159, Loss: 4.77051305770874, Accuracy: 0.0478515625
Batch: 160, Loss: 5.037728786468506, Accuracy: 0.052734375
Batch: 161, Loss: 5.038655757904053, Accuracy: 0.0654296875
Batch: 162, Loss: 5.042209625244141, Accuracy: 0.044921875
Batch: 163, Loss: 4.881163597106934, Accuracy: 0.0556640625
Batch: 164, Loss: 5.143684387207031, Accuracy: 0.0400390625
Batch: 165, Loss: 5.109106063842773, Accuracy: 0.0576171875
Batch: 166, Loss: 5.057214260101318, Accuracy: 0.0361328125
Batch: 167, Loss: 4.978276252746582, Accuracy: 0.03515625
Batch: 168, Loss: 5.199067115783691, Accuracy: 0.0361328125
Batch: 169, Loss: 4.991067886352539, Accuracy: 0.0712890625
Batch: 170, Loss: 5.204485893249512, Accuracy: 0.0

Batch: 293, Loss: 4.909258842468262, Accuracy: 0.0419921875
Batch: 294, Loss: 4.892879962921143, Accuracy: 0.0498046875
Batch: 295, Loss: 4.973970890045166, Accuracy: 0.0322265625
Batch: 296, Loss: 4.969633102416992, Accuracy: 0.0302734375
Batch: 297, Loss: 4.889212608337402, Accuracy: 0.0283203125
Batch: 298, Loss: 4.855247497558594, Accuracy: 0.015625
Batch: 299, Loss: 4.811577796936035, Accuracy: 0.03515625
Batch: 300, Loss: 4.7580108642578125, Accuracy: 0.0341796875
Batch: 301, Loss: 4.934909343719482, Accuracy: 0.04296875
Batch: 302, Loss: 4.842254638671875, Accuracy: 0.048828125
Batch: 303, Loss: 4.905360221862793, Accuracy: 0.0498046875
Batch: 304, Loss: 4.942501068115234, Accuracy: 0.0595703125
Batch: 305, Loss: 4.816242694854736, Accuracy: 0.0732421875
Batch: 306, Loss: 4.790748596191406, Accuracy: 0.07421875
Batch: 307, Loss: 4.735777378082275, Accuracy: 0.0712890625
Batch: 308, Loss: 4.729455471038818, Accuracy: 0.0458984375
Batch: 309, Loss: 4.622993469238281, Accuracy: 0.0

Batch: 76, Loss: 4.721621513366699, Accuracy: 0.044921875
Batch: 77, Loss: 4.594506740570068, Accuracy: 0.08203125
Batch: 78, Loss: 4.625, Accuracy: 0.0830078125
Batch: 79, Loss: 4.615630149841309, Accuracy: 0.0712890625
Batch: 80, Loss: 4.603630065917969, Accuracy: 0.0439453125
Batch: 81, Loss: 4.682187557220459, Accuracy: 0.0556640625
Batch: 82, Loss: 4.675192356109619, Accuracy: 0.0654296875
Batch: 83, Loss: 4.697556495666504, Accuracy: 0.060546875
Batch: 84, Loss: 4.83424186706543, Accuracy: 0.041015625
Batch: 85, Loss: 4.8084306716918945, Accuracy: 0.0654296875
Batch: 86, Loss: 4.787200927734375, Accuracy: 0.072265625
Batch: 87, Loss: 4.816173076629639, Accuracy: 0.0546875
Batch: 88, Loss: 4.719335556030273, Accuracy: 0.056640625
Batch: 89, Loss: 4.623489856719971, Accuracy: 0.048828125
Batch: 90, Loss: 4.7606730461120605, Accuracy: 0.048828125
Batch: 91, Loss: 4.6194610595703125, Accuracy: 0.037109375
Batch: 92, Loss: 4.548745155334473, Accuracy: 0.0537109375
Batch: 93, Loss: 4.4

Batch: 216, Loss: 5.08053731918335, Accuracy: 0.1015625
Batch: 217, Loss: 5.030010223388672, Accuracy: 0.1123046875
Batch: 218, Loss: 5.025328636169434, Accuracy: 0.1201171875
Batch: 219, Loss: 5.093874931335449, Accuracy: 0.115234375
Batch: 220, Loss: 4.934689998626709, Accuracy: 0.109375
Batch: 221, Loss: 4.8603010177612305, Accuracy: 0.109375
Batch: 222, Loss: 5.177408218383789, Accuracy: 0.0830078125
Batch: 223, Loss: 5.291857719421387, Accuracy: 0.0791015625
Batch: 224, Loss: 5.146108150482178, Accuracy: 0.083984375
Batch: 225, Loss: 5.048740386962891, Accuracy: 0.08984375
Batch: 226, Loss: 5.142303943634033, Accuracy: 0.08203125
Batch: 227, Loss: 5.030485153198242, Accuracy: 0.0771484375
Batch: 228, Loss: 5.063514709472656, Accuracy: 0.0791015625
Batch: 229, Loss: 4.933019638061523, Accuracy: 0.0751953125
Batch: 230, Loss: 5.020380973815918, Accuracy: 0.0859375
Batch: 231, Loss: 5.077701568603516, Accuracy: 0.087890625
Batch: 232, Loss: 4.935580730438232, Accuracy: 0.080078125
Ba

Batch: 355, Loss: 4.62380313873291, Accuracy: 0.0205078125
Batch: 356, Loss: 4.671951770782471, Accuracy: 0.0185546875
Batch: 357, Loss: 4.693922996520996, Accuracy: 0.015625
Batch: 358, Loss: 4.887599468231201, Accuracy: 0.0166015625
Saved Weights at epoch 80 to file Weights_80.h5


## Generating music (In progress)

In [64]:
def make_model(unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 1024, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
    #remember, that here we haven't given return_sequences = True because here we will give only one character to generate the
    #sequence. In the end, we just have to get one output which is equivalent to getting output at the last time-stamp. So, here
    #in last layer there is no need of giving return sequences = True.
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [65]:
def generate_sequence(epoch_num, initial_index, seq_length):
    with open(charIndex_json) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    model.load_weights(model_weights_directory + "Test3_Weights_{}.h5".format(epoch_num))
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
    #above code is for ignoring the starting string of a generated sequence. This is because we are passing any arbitrary 
    #character to the model for generating music. Now, the model start generating sequence from that character itself which we 
    #have passed, so first few characters before "\n" contains meaningless word. Model start generating the music rhythm from
    #next line onwards. The correct sequence it start generating from next line onwards which we are considering.
    
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    seq2 = seq1[:cnt]
    #Now our data contains three newline characters after every tune. So, the model has leart that too. So, above code is used for
    #ignoring all the characters that model has generated after three new line characters. So, here we are considering only one
    #tune of music at a time and finally we are returning it..
    
    return seq2

In [66]:
ep = int(input("1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: "))
ar = int(input("\n2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: "))
ln = int(input("\n3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: "))

music = generate_sequence(ep, ar, ln)

print("\nMUSIC SEQUENCE GENERATED: \n")

print(music)

1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: 80

2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: 67

3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: 550


ValueError: Dimension 1 in both shapes must be equal, but are 1024 and 512. Shapes are [1028,1024] and [1028,512]. for 'Assign_5' (op: 'Assign') with input shapes: [1028,1024], [1028,512].